***XG BOOST / GRADIENT BOOSTING - Kaggle Competitions Dominator*** 


Ta đã được học về Random Forest - 1 ensemble model xây dựng dựa trên Decision Tree. Về cơ bản, ensemble model là tổng hợp kết quả trung bình của các model con.  
XG Boost chính là 1 ensemble   

**XGBoost là một thuật toán boosting: bắt đầu từ dự đoán gốc, rồi lần lượt thêm các cây quyết định. Mỗi cây mới học trên gradient và hessian (đạo hàm bậc 1 v2 2) của hàm mất mát (loss function - tương tự MAE) ở bước trước, nhờ đó giảm dần sai số qua từng vòng lặp. Các tham số do người dùng đặt sẽ quyết định cách mô hình cập nhật, tốc độ hội tụ và khả năng tránh overfitting.**


In [116]:
import pandas as pd
import numpy as np

In [117]:
raw_train = pd.read_csv('./train.csv')
raw_test = pd.read_csv('./test.csv')
y = raw_train['SalePrice']

In [118]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [119]:
train = raw_train.copy()
train.drop('SalePrice', axis = 1, inplace = True)

In [120]:
num_col = train.select_dtypes(include = ['float', 'int'])
cat_col = train.select_dtypes(include = 'object')

In [121]:
num_col = num_col.columns
cat_col = cat_col.columns

Setup các *Preprocessing Method*  
***Pipeline*** : impute và onehot cho các cột categorical  
  Pipeline sẽ đi từng bước trong steps -> imputer -> onehot

***CT*** : nếu Pipeline là gom các preprocessing method thành bundle, thì CT sẽ gom các cột cần preprocessed thành bundle

In [122]:
numerical_pp = SimpleImputer(strategy='mean')
categorical_pp = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
preprocessor = ColumnTransformer(
    transformers = [
    ('num', numerical_pp, num_col),
    ('cate', categorical_pp, cat_col)
], remainder='passthrough')

In [123]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train, y)

In [124]:
from sklearn.ensemble import RandomForestRegressor
RFR = RandomForestRegressor(random_state=1, max_leaf_nodes=250, max_depth=7)

Output của pipeline categorical_PP là transformed data. Đây còn gọi là sub-pipeline

Output của PLP là prediction của model : preprocessing -> model. Đây là end-to-end pipeline

In [125]:
PLP = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(max_leaf_nodes=13, max_depth=10))
])
PLP.fit(x_train, y_train) 
PLP_pred = PLP.predict(x_val)


Import XGBoost  
Trong xgboost phân ra các model : XGB - Regressor/Classifier : XGB cơ bản  
                                  XGBRF : Random Forest dùng thuật toán XGB


split training set để tạo train subset và validation subset (phục vụ cho early stopping round và eval set)

In [126]:
x_val.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
458,459,70,RM,NaN,5100,Pave,Grvl,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,6,2008,WD,Normal
1123,1124,20,RL,50.0,9405,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal
513,514,20,RL,71.0,9187,Pave,NaN,Reg,Bnk,AllPub,...,0,0,NaN,NaN,NaN,0,6,2007,WD,Normal
1439,1440,60,RL,80.0,11584,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
957,958,20,RL,70.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2007,WD,Normal


Preprocess train trước (không dùng PLP thì phải thế)

In [127]:
x_train = pd.DataFrame(preprocessor.fit_transform(x_train))

In [128]:
x_val = pd.DataFrame(preprocessor.transform(x_val))

Tuning cho Model :
**n_estimators** : số lượng fitting cycles  

**early_stopping_rounds** : khi loss function sau các cycles có dấu hiệu chững lại, ESR cho phép dừng sớm việc 
fit. early_stopping_rounds = 5 là lý tưởng : nếu sau 5 cycles liên tục chững lại thì dừng fit   

**eval_set** : là parameter đi kèm với *e_s_r*, đầu vào là [x_val, y_val]. parameter này là cơ sở để giám sát esr, nếu không có eval_set, loss function gần như sẽ luôn giảm, không có cơ sở để đánh giá và dừng fit của esr  

**verbose** : in ra info log cho mỗi cycle : 0 : không in, 1 : in cơ bản, 2 : in chi tiết  

**learning_rate** : (tên tham số: eta) là hệ số nhân với đóng góp của mỗi cây mới vào mô hình tổng. eta (def_val = 0,1) là hệ số đóng góp của mỗi component vào model  



In [1]:
from xgboost import XGBRegressor

XGBR = XGBRegressor(
    n_estimators=250,
    eta=0.1,
    eval_metric="mae",
    early_stopping_rounds=5
)

XGBR.fit(
    x_train, y_train,
    eval_set=[(x_val, y_val)],
    verbose=0
)
scores = XGBR.predict(x_val)


NameError: name 'x_train' is not defined

In [130]:
from sklearn.metrics import mean_absolute_error 
accuracy2 = mean_absolute_error(PLP_pred, y_val)
accuracy1 = mean_absolute_error(scores, y_val) 
print(f"PLP : {accuracy2}, XGB : {accuracy1}")

PLP : 24993.967341892007, XGB : 18472.76171875


In [131]:
import xgboost
print(xgboost.__version__)

2.0.0
